This notebook creates the dataframe for city for food and attractions. All work is duplicated in working user-city-sim notebook

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import ast
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def get_df(table_name):
    try:
        conn = sqlite3.connect('/Users/tristannisbet/Documents/travel_app/places.db')

    except Exception as e:
        print('Error durring connection: ', str(e))
    
    sql = """select * from {}""".format(table_name)
    df = pd.read_sql_query(sql, conn)

    return df

In [ ]:
one = get_df('restaurants_one')
two = get_df('restaurants_two')
three = get_df('restaurants_three')
four = get_df('restaurants_four')
top_rest = get_df('restaurants')


In [ ]:
all_food = pd.concat([one, two, three, four, top_rest], axis =0)

In [ ]:
all_food.info()

In [ ]:
def to_city(df):
    city_df = df.groupby(['country', 'city', 'id', 'price_level'])['name'].count().to_frame()
    price_level = city_df.pivot_table(index=['country', 'city', 'id'], columns='price_level', values='name', aggfunc='first')
    price_level['avg_price'] = df.groupby(['country', 'city', 'id'])['price_level'].mean()

    
    return price_level


In [ ]:
test = to_city(all_food)
test

In [ ]:
test.iloc[test.index.get_level_values('id') == "3"]

In [ ]:
attractions = get_df('attractions')
attractions.sample(5)

In [ ]:
place_of_worship = ['place_of_worship', 'hindu_temple', 'church', 'mosque', 'synagogue']
shopping = ['store', 'shopping_mall', 'clothing_store', 'electronics_store', 'grocery_or_supermarket', 'department_store']

attractions_to_keep = ['amusement_park', 'museum', 'park', 'art_gallery', 'aquarium',
                      'zoo', 'library', 'movie_theater', 'natural_feature'] + place_of_worship + shopping

attractions_to_keep


In [ ]:

def main_call(attraction_df):
    attractions_split = split_types(attraction_df)
    dummy = dummies(attractions_split)
    by_city, all_attractions = attraction_count(dummy, attractions_split)
    
    return by_city, all_attractions

def split_types(df):
    df['split_types'] = [ast.literal_eval(x) for x in df.types]
    df['split_types_str'] = [','.join(x) for x in df.split_types]
    
    return df

def dummies(df):
    dummies = df.split_types_str.str.get_dummies(sep=',')

    return dummies

def attraction_count(dummies_df, all_attractions_df):
    all_attractions_df = pd.concat([all_attractions_df, dummies_df], axis=1)
    type_col_names = attractions_to_keep
    type_col_names.extend(['country', 'city', 'id'])
    attraction_count = all_attractions_df[type_col_names].groupby(['country', 'city', 'id']).sum()
    
    return attraction_count, all_attractions_df

In [ ]:
#city, all_attractions = main_call(attractions)

In [ ]:
attractions_split = split_types(attractions)
attractions_split

In [ ]:
df = dummies(attractions_split)
df

In [ ]:
df2, all_attractions = attraction_count(df, attractions_split)

In [ ]:
df2

In [ ]:
all_attractions

In [ ]:
tourist_types = attractions_split['split_types'].apply(pd.Series).stack().unique()
len(tourist_types)

In [ ]:
for t in tourist_types:
    print(t)

In [ ]:
all_attractions

In [ ]:
# How many missing attractions are not included. (about 100)
left_out = ['country', 'city','school', 'stadium', 'gym', 'casino', 'bowling_alley', 'night_club',
           'spa', 'beauty_salon', 'home_goods_store', 'book_store', 'university', 'campground']

df2['left_out'] =  all_attractions[left_out].groupby(['country', 'city']).sum().sum(axis=1)


In [ ]:
city = pd.merge(test, df2, on = ['country', 'city', 'id'], how = 'outer')
city

In [ ]:
city.reset_index(inplace=True)
city['id'] = pd.to_numeric(city['id'])

In [ ]:
city.set_index(['country', 'city', 'id'], inplace=True)
city.sort_values('id', inplace=True)
city

In [ ]:
city.iloc[city.index.get_level_values('city') == "Sydney"]

In [ ]:
# Filling null values so cosine will work
city[city[1.0].isnull()]
city.fillna(0, inplace=True)

In [ ]:
cosine_city = pd.DataFrame(cosine_similarity(city))

cosine_city

In [ ]:
def find_similar_n(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [ ]:
top_10 = find_similar_n(cosine_city, 10)
top_10

In [ ]:
top_10.iloc[49]

In [ ]:
city.iloc[[54, 10, 102, 52,71,99,106,4,33,13]]

In [ ]:
city.iloc[[49, 123, 120, 9, 59, 73, 87, 61, 115, 82]]

In [ ]:
dublin = city.iloc[[54, 84, 101, 110, 4, 81, 100, 42, 92, 80]].copy()

In [ ]:
dublin.iloc[:, 5:]

In [ ]:
dublic.iloc

In [ ]:
#WITHOUT SORTING INDEX?m
city.iloc[[0,85,5,23,42,10,100,103,1,8]]

In [ ]:
city.iloc[[0,7,13,102,2,33,71,52,106,1]]

In [ ]:
city.iloc[[134,78,132,107,30,29,39,33,62,7]]

In [ ]:
one =  city.iloc[city.index.get_level_values('city') == "Sydney"]
two = city.iloc[city.index.get_level_values('city') == "London"]
three = city.iloc[city.index.get_level_values('city') == "Dublin"]

In [ ]:
one

In [ ]:
two

In [ ]:
three

In [ ]:
city.iloc[0]